In [84]:
!pip install periodictable


  Stored in directory: C:\Users\zachmih\AppData\Local\pip\Cache\wheels\13\4a\aa\62f2e31825986256396d88f55d7b5144db5d4e60a4573de958
Successfully built periodictable


You are using pip version 19.0.3, however version 19.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
import re

In [56]:
# read in the various CSVs from the scraping

raw_6 = pd.read_csv('Output Data/Articles_6.csv', dtype = str )

raw_6["Stats"]

0       EA Friis, RS Lakes, JB Park - Journal of Mater...
1       XH Yang, JX Bai, HB Yan, JJ Kuang, TJ Lu… - Tr...
2       LE Murr, SM Gaytan, F Medina, E Martinez… - Ma...
3       EM Castrodeza, C Mapelli, M Vedani… - Journal ...
4       Z Wang, H Ma, L Zhao, G Yang - Scripta Materia...
5       M Wicklein, K Thoma - Materials Science and En...
6       H Liu, J Wei, Z Qu - Journal of Sound and Vibr...
7       S De Schampheleire, P De Jaeger, K De Kerpel… ...
8       N Michailidis, F Stergioudi, H Omar, DN Tsipas...
9       DJ Sypeck, HNG Wadley - Advanced Engineering M...
10      EA Friis, RS Lakes, JB Park - Journal of Mater...
11      XH Yang, JX Bai, HB Yan, JJ Kuang, TJ Lu… - Tr...
12      LE Murr, SM Gaytan, F Medina, E Martinez… - Ma...
13      EM Castrodeza, C Mapelli, M Vedani… - Journal ...
14      Z Wang, H Ma, L Zhao, G Yang - Scripta Materia...
15      M Wicklein, K Thoma - Materials Science and En...
16      H Liu, J Wei, Z Qu - Journal of Sound and Vibr...
17      S De S

In [3]:
raw = pd.read_csv('Output Data/Articles.csv', dtype=str)
raw.head()

,Authors,Citations,Date,Full Text Available,Journal,Stats,Summary,Title
0,"TJ Lu, HA Stone, MF Ashb",567,1998,[PDF] paper.edu.cn,Acta materialia,"TJ Lu, HA Stone, MF Ashby - Acta materialia, 1...",The paper explores the use of open-celled meta...,Heat transfer in open-cell metal foams
1,"K Boomsma, D Poulikakos, Y Ventiko",295,2003,[PDF] psu.edu,… Journal of Heat and Fluid Flow,"K Boomsma, D Poulikakos, Y Ventikos - … Journa...",A new approach to modeling the flow through a ...,Simulations of flow through open cell metal fo...
2,"S Krishnan, JY Murthy",198,2006,[PDF] purdue.edu,Journal of …,"S Krishnan, JY Murthy… - Journal of …, 2006 - ...",Flows in porous media may be modeled using two...,Direct simulation of transport in open-cell me...
3,"A Kopanidis, A Theodorakakos, E Gavaises",141,2010,[PDF] academia.edu,International Journal of …,"A Kopanidis, A Theodorakakos, E Gavaises… - In...",A 3D numerical simulation methodology for the ...,3D numerical simulation of flow and conjugate ...
4,"YW Kwon, RE Cooke, C Par",119,2003,No,Materials Science and Engineering: A,"YW Kwon, RE Cooke, C Park - Materials Science ...",Representative unit-cell models were developed...,Representative unit-cell models for open-cell ...


In [4]:
raw_3 = pd.read_csv('Output Data/Articles_3.csv', dtype=str)
raw_3.head()

,Authors,Citations,Date,Full Text Available,Journal,Stats,Summary,Title
0,"L Jing, Z Wang, J Ning, L Zha",73,2011,No,Composites Part B: Engineering,"L Jing, Z Wang, J Ning, L Zhao - Composites Pa...",The deformation and failure modes of dynamical...,The dynamic response of sandwich beams with op...
1,"KC Leong, LW Ji",65,2006,No,International Journal of Heat and fluid flow,"KC Leong, LW Jin - International Journal of He...",An experimental study was performed to investi...,Characteristics of oscillating flow through a ...
2,"WY Jang, S Kyriakides, AM Krayni",102,2010,[HTML] sciencedirect.com,International Journal of Solids and …,"WY Jang, S Kyriakides, AM Kraynik - Internatio...",Two families of finite element models of aniso...,[HTML][HTML] On the compressive strength of op...
3,"X Xiao, P Zhang, M L",91,2014,No,International Journal of Thermal Sciences,"X Xiao, P Zhang, M Li - International Journal ...",The thermal conductivity of phase change mater...,Effective thermal conductivity of open-cell me...
4,"KP Dharmasena, HNG Wadle",78,2002,[PDF] virginia.edu,Journal of materials research,"KP Dharmasena, HNG Wadley - Journal of materia...",Cellular metal foams are of interest because o...,Electrical conductivity of open-cell metal foams


In [85]:
## Something wrong with this csv
raw_5 = pd.read_csv(Output Data/Articles_5.csv')
raw_5.head()


SyntaxError: invalid syntax (<ipython-input-85-e8094358f1ab>, line 2)

## Something wrong with this csv 
raw_4 = pd.read_csv('Project-2/Output Data/Articles_4.csv')
raw_4.head()

# Concatenate the csvs that read well and then start cleaning the data

1. Remove duplicate rows
2. Reset the index
3. Remove rows which have non-numerical data in the Citations column
4. Re-extract the "Date"  from the "Stats" column (some cells were corrupted),  and then change the datatype to int
5. Create Topics_Tags column by parsing the Summary text and Title text and comparing to a list of Topics keywords
6. Create Properties_Tags column by parsing the Summary text and Title text and comparing to a list of Mechanical and Thermal properties keywords
7. Nice to have:  Re-extract the "Authors" and "Journals"  from the Stats column;  there are some spelling errors.

In [5]:
len(raw), len(raw_3), len(raw_6)

(10, 20, 2611)

In [6]:
# Concatenate the csvs that seem to be working

references = pd.concat([raw,raw_3,raw_6])



In [7]:
#ref = references.astype({"Authors": str, "Citations": str, "Date": str, "Full Text Available": str,
#                         "Journal": str, "Stats": str, "Summary": str, "Title": str,
#                         })

In [8]:
# Remove duplicate rows

drop_dup = references.drop_duplicates()

len(drop_dup), len(references)

(849, 2641)

In [9]:
# reset the index for ref

ref  = drop_dup.reset_index(drop=True)
ref.head(10)

,Authors,Citations,Date,Full Text Available,Journal,Stats,Summary,Title
0,"TJ Lu, HA Stone, MF Ashb",567,1998,[PDF] paper.edu.cn,Acta materialia,"TJ Lu, HA Stone, MF Ashby - Acta materialia, 1...",The paper explores the use of open-celled meta...,Heat transfer in open-cell metal foams
1,"K Boomsma, D Poulikakos, Y Ventiko",295,2003,[PDF] psu.edu,… Journal of Heat and Fluid Flow,"K Boomsma, D Poulikakos, Y Ventikos - … Journa...",A new approach to modeling the flow through a ...,Simulations of flow through open cell metal fo...
2,"S Krishnan, JY Murthy",198,2006,[PDF] purdue.edu,Journal of …,"S Krishnan, JY Murthy… - Journal of …, 2006 - ...",Flows in porous media may be modeled using two...,Direct simulation of transport in open-cell me...
3,"A Kopanidis, A Theodorakakos, E Gavaises",141,2010,[PDF] academia.edu,International Journal of …,"A Kopanidis, A Theodorakakos, E Gavaises… - In...",A 3D numerical simulation methodology for the ...,3D numerical simulation of flow and conjugate ...
4,"YW Kwon, RE Cooke, C Par",119,2003,No,Materials Science and Engineering: A,"YW Kwon, RE Cooke, C Park - Materials Science ...",Representative unit-cell models were developed...,Representative unit-cell models for open-cell ...
5,"N Dukhan, PD Quinon",102,E Cru,[PDF] academia.edu,amos,"N Dukhan, PD Quinones-Ramos, E Cruz-Ruiz… - In...",A one-dimensional heat transfer model for open...,One-dimensional heat transfer analysis in open...
6,"DT Queheillalt, DD Hass, DJ Sypeck",93,2001,[PDF] 128.143.22.36,Journal of Materials …,"DT Queheillalt, DD Hass, DJ Sypeck… - Journal ...","Low-density, open-cell nickel base superalloy ...",Synthesis of open-cell metal foams by template...
7,"B Ozmat, B Leyda, B Benso",82,2004,[PDF] metallicfoams.com,Materials and manufacturing …,"B Ozmat, B Leyda, B Benson - Materials and man...",The key structural and thermo-physical propert...,Thermal applications of open-cell metal foams
8,"M Bai, JN Chun",99,2011,No,International Journal of Thermal Sciences,"M Bai, JN Chung - International Journal of The...",Enhanced cooling methods are needed for advanc...,Analytical and numerical prediction of heat tr...
9,"KK Bodla, JY Murthy, SV Garimell",91,"Part …, 2010",[PDF] purdue.edu,Numerical Heat Transfer,"KK Bodla, JY Murthy, SV Garimella - Numerical ...",Important heat transfer parameters of aluminum...,Microtomography-based simulation of transport ...


# Eliminate non-numerics from the Citation Column


In [10]:
# Drop Rows based on whether or not the Citation column contains "% R",  although I couldn't make the wildcards work in this method.
# The below methods were tried and don't work
## STUCK HERE - Trying to eliminate the rows where the Citation number is corrupted (i.e. not a number)
## The pattern which we are trying to get rid of is "4 R", "5 R", "6 R", "7 R", "8 R", "9 R"  in the citation column

# None of the below approaches worked 
#refs_2 = refs_2.drop(refs_1[refs_2.Citations.contains(' R', regex=True)].index)
#refs_2[refs_2.Citations.str.contains('^R'), :] 
#refs_2.loc[refs_2.Citations.str.contains(' R', regex = True), :]

# I was also planning to eliminate the US Patents, from the "Journal" column,  it's more tedious than expected
# https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression

#ref1 = ref.drop(ref[ref.Journal=="US Patent 4"].index)
#ref2 = ref1.drop(ref1[ref1.Journal=="US Patent 5"].index)
#ref3 = ref2.drop(ref2[ref2.Journal=="US Patent 6"].index)
#ref4 = ref3.drop(ref3[ref3.Journal=="US Patent 7"].index)
#ref5 = ref4.drop(ref4[ref4.Journal=="US Patent 8"].index)
#ref6 = ref5.drop(ref5[ref5.Journal=="US Patent 9"].index)
#ref7 = ref6.drop(ref6[ref6.Journal=="US Patent"].index)


ref1 = ref.drop(ref[ref.Citations=="4 R"].index)
ref2 = ref1.drop(ref1[ref1.Citations=="5 R"].index)
ref3 = ref2.drop(ref2[ref2.Citations=="6 R"].index)
ref4 = ref3.drop(ref3[ref3.Citations=="7 R"].index)
ref5 = ref4.drop(ref4[ref4.Citations=="8 R"].index)
ref6 = ref5.drop(ref5[ref5.Citations=="9 R"].index)

#len(ref6)

ref6  = ref6.reset_index(drop=True)
ref6["Stats"]






0      TJ Lu, HA Stone, MF Ashby - Acta materialia, 1...
1      K Boomsma, D Poulikakos, Y Ventikos - … Journa...
2      S Krishnan, JY Murthy… - Journal of …, 2006 - ...
3      A Kopanidis, A Theodorakakos, E Gavaises… - In...
4      YW Kwon, RE Cooke, C Park - Materials Science ...
5      N Dukhan, PD Quinones-Ramos, E Cruz-Ruiz… - In...
6      DT Queheillalt, DD Hass, DJ Sypeck… - Journal ...
7      B Ozmat, B Leyda, B Benson - Materials and man...
8      M Bai, JN Chung - International Journal of The...
9      KK Bodla, JY Murthy, SV Garimella - Numerical ...
10     L Jing, Z Wang, J Ning, L Zhao - Composites Pa...
11     KC Leong, LW Jin - International Journal of He...
12     WY Jang, S Kyriakides, AM Kraynik - Internatio...
13     X Xiao, P Zhang, M Li - International Journal ...
14     KP Dharmasena, HNG Wadley - Journal of materia...
15     M Vesenjak, C Veyhl, T Fiedler - Materials Sci...
16     DT Queheillalt, Y Katsumura, HNG Wadley - Scri...
17     S Krishnan, SV Garimella

In [11]:
# Change datatype for Citations column

ref7 = ref6.astype({"Citations": int, })


In [12]:
len(ref7), ref7.dtypes
#we have dropped out 48 references

(801, Authors                object
 Citations               int32
 Date                   object
 Full Text Available    object
 Journal                object
 Stats                  object
 Summary                object
 Title                  object
 dtype: object)


# Year from the Stats column and change the datatype to integer

There are some errors in the columns,  for Author names, year and journal
https://chrisalbon.com/python/data_wrangling/pandas_regex_to_create_columns/

https://www.geeksforgeeks.org/split-a-string-into-columns-using-regex-in-pandas-dataframe/

In [13]:
# Clean up the Date column  and make the dtype integer

ref7['Date'] = ref7['Stats'].str.extract('(\d\d\d\d)', expand=True) 

ref8 = ref7.astype({"Date": int, })

ref8.dtypes




Authors                object
Citations               int32
Date                    int32
Full Text Available    object
Journal                object
Stats                  object
Summary                object
Title                  object
dtype: object

In [14]:
# Use Python Regex to get the Journal name cleanly from the Stats column
# Also try to get the Authors names cleanly
ref8["Stats"].head(), len(ref8)


(0    TJ Lu, HA Stone, MF Ashby - Acta materialia, 1...
 1    K Boomsma, D Poulikakos, Y Ventikos - … Journa...
 2    S Krishnan, JY Murthy… - Journal of …, 2006 - ...
 3    A Kopanidis, A Theodorakakos, E Gavaises… - In...
 4    YW Kwon, RE Cooke, C Park - Materials Science ...
 Name: Stats, dtype: object, 801)

In [43]:
# The regex doesn't work the way I expect,  likely it's due to the extra characters one can see in the CSV file
# Somtimes gives NaNs

# use regex with pandas to extract the Publisher data better....still having trouble with web address

#journalRegex = re.compile(r'- (\w.*)\,')   
# Test code for the syntax


ref8['Journals'] = ref8['Stats'].str.extract(r'- (\w.*)\,')
ref8['Journals'].head(20), len(ref8)

#ref8['Publisher'] = ref8['Stats'].str.extract(r'(- (w.*)\, | - ([a-zA-Z0-9_.+-]+))')
# Potentially add publisher information .  This extraction seems to get some of the web address

# Used the first reference
# https://stackoverflow.com/questions/49184752/iterate-different-regular-expression-patterns-over-df-column
#  https://stackoverflow.com/questions/49169725/combining-for-loop-iterations-into-a-single-line-and-no-match-handling

(0                                     Acta materialia
 1                                                 NaN
 2                                        Journal of …
 3                          International Journal of …
 4                Materials Science and Engineering: A
 5                          International Journal of …
 6                              Journal of Materials …
 7                       Materials and manufacturing …
 8           International Journal of Thermal Sciences
 9                     Numerical Heat Transfer, Part …
 10                     Composites Part B: Engineering
 11       International Journal of Heat and fluid flow
 12              International Journal of Solids and …
 13          International Journal of Thermal Sciences
 14                      Journal of materials research
 15               Materials Science and Engineering: A
 16                                 Scripta Materialia
 17                                       Journal of …
 18    Int

In [50]:
# The regex doesn't work the way I expect,  likely it's due to the extra characters one can see in the CSV file
# Extract the first author name only,  sometimes gives NaNs
ref8['First Author'] = ref8['Authors'].str.extract(r'(\w.*?)((\,) | (\ -))')
ref8['First Author'].tail(20), len(ref8)


(781                Y Mu
 782              C Tuck
 783               ZX Lu
 784               L Jin
 785                 NaN
 786                Y Li
 787               Y Sun
 788            YA Samad
 789            H Eifert
 790         JM Thompson
 791            C Heisig
 792              L Gong
 793          PK Pinnoji
 794           A Burteau
 795                 NaN
 796             C Kádár
 797    J Randrianalisoa
 798                 NaN
 799              W Yang
 800           KC Kelley
 Name: First Author, dtype: object, 801)

# Create lists of keywords to compare to the abstract and title

In [ ]:
mech_properties = []
thermal_props = []
elements = []
element_names = []
mech_tags = []
thermal_tags = []

# Create Topic Tags by searching for adjectives and periodic table elements in front of Open cell Metal Foam in the summary and title


# Prepare the Journal and Author data for plotting in Chart.js

In [79]:
journal_pareto = ref8["Journals"].value_counts()
#journal_pareto.head(10)
journals = journal_pareto.iloc[:10,]
#journals



Materials & Design                          36
Materials Science and Engineering: A        32
Materials Science and …                     27
International Journal of …                  25
Scripta Materialia                          20
Acta Materialia                             17
International Journal of Heat and Mass …    16
Journal of …                                15
Materials Letters                           13
Advanced …                                  13
Name: Journals, dtype: int64

In [81]:
Author_pareto = ref8["First Author"].value_counts()
#Author_pareto. head(10)
authors = Author_pareto[:10,]
authors

N Dukhan         16
N Michailidis     7
A Jung            7
CY Zhao           6
S Mancin          6
MAA Mendes        6
X Xia             5
E Amsterdam       5
DP Mondal         4
P De Jaeger       4
Name: First Author, dtype: int64

In [82]:
ref8.to_csv("Output Data/reference.csv")
references.to_csv("Output Data/ref_with_dups.csv")
journals.to_csv("Output Data/journals.csv")
authors.to_csv("Output Data/authors.csv")
